In [28]:
##### ETL NOTEBOOK FOR 2023 MHSAA TOURNEY SPECIFIC MAP

#### Adapted from ETL for JSON

## Dependencies and Setup
### Dependencies

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import re
import time

## Start timer
start_time = time.time()


In [29]:
## LOAD BLOCK###
#### Load data from kml file exported from Google Earth

file_path = ('data/kml/MHSAA_2023.kml') # file path to kml file


# Read the KML file
with open(file_path) as file:
    xml_data = file.read()

# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')
folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')

# Create a list to store rows to append to the DataFrame
rows = []

# Loop through the folders and extract the data
for folder in list:
    try:
        field_name = folder.find('name').text
        foul = folder.find_all('coordinates')[0].text
        fop = folder.find_all('coordinates')[1].text
        notes = None

        # Check if there is a description tag, if so, use it for notes
        if folder.find('description') is not None:
            notes = folder.find('description').text

        row = {
            'field': field_name,
            'foul': foul,
            'fop': fop,
            'notes': notes
        }

        rows.append(row)

    except Exception as e:
        # Add name of folder to a list of failed folders
        failed.append(folder.find('name').text)
        print(f"Error processing folder: {folder.find('name').text}. Error message: {str(e)}")

# Convert the list of rows to a DataFrame
df = pd.DataFrame(rows, columns=['field', 'foul', 'fop', 'notes'])

# print('Failed to parse:', failed)


In [30]:
df.head()

,field,foul,fop,notes
0,Indian River Inland Lakes HS,"\n\t\t\t\t\t\t\t\t-84.6358958,45.3838713,0 -84...","\n\t\t\t\t\t\t\t\t-84.6358958,45.3838713,0 -84...",None
1,Holton HS,"\n\t\t\t\t\t\t\t\t-86.0881713,43.4149801,0 -86...","\n\t\t\t\t\t\t\t\t-86.0881713,43.4149801,0 -86...",None
2,Jackson Lumen Christi HS,"\n\t\t\t\t\t\t\t\t-84.4602665,42.2219816,0 -84...","\n\t\t\t\t\t\t\t\t-84.4602665,42.2219816,0 -84...",None
3,Cornerstone Baseball Field - college,"\n\t\t\t\t\t\t\t\t-85.5975511,42.9811225,0 -85...","\n\t\t\t\t\t\t\t\t-85.5975511,42.9811225,0 -85...",None
4,Adrian College,"\n\t\t\t\t\t\t\t\t-84.0697145,41.901861,0 -84....","\n\t\t\t\t\t\t\t\t-84.0697145,41.901861,0 -84....",None


In [31]:
# Clean the new dataframe


# Create a copy of the original DataFrame
df_cleaned = df.copy()

# Remove new line and space characters from coordinates
df_cleaned = df_cleaned.replace(r'\n','', regex=True) 
df_cleaned = df_cleaned.replace(r'\t','', regex=True) 

# Drop any duplicate rows
df_cleaned = df_cleaned.drop_duplicates(subset=['field'], keep='first')

# Drop any rows with empty fields
df_cleaned = df_cleaned[(df_cleaned != 0).all(1)]

In [32]:
df_cleaned.info()
df_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 145
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   field   142 non-null    object
 1   foul    142 non-null    object
 2   fop     142 non-null    object
 3   notes   7 non-null      object
dtypes: object(4)
memory usage: 5.5+ KB


,field,foul,fop,notes
0,Indian River Inland Lakes HS,"-84.6358958,45.3838713,0 -84.6346834,45.383949...","-84.6358958,45.3838713,0 -84.63578579999999,45...",None
1,Holton HS,"-86.0881713,43.4149801,0 -86.0887185,43.415773...","-86.0881713,43.4149801,0 -86.0870568,43.415367...",None
2,Jackson Lumen Christi HS,"-84.4602665,42.2219816,0 -84.4603248,42.222842...","-84.4602665,42.2219816,0 -84.4591654,42.222017...",None
3,Cornerstone Baseball Field - college,"-85.5975511,42.9811225,0 -85.5980533,42.981930...","-85.5975511,42.9811225,0 -85.59644400000001,42...",None
4,Adrian College,"-84.0697145,41.901861,0 -84.0703958,41.9026485...","-84.0697145,41.901861,0 -84.0687248,41.9023461...",None


In [33]:
##### Clean up polygon data and create a new home_plate column

def parse_coordinates(coord_string):
    coords = coord_string.split()
    parsed_coords = [tuple(map(float, coord.split(',')[:2])) for coord in coords]
    return parsed_coords

# Create a new column for the home_plate location using the first set of coordinates in the 'fop' column
df_cleaned['home_plate'] = df_cleaned['fop'].apply(lambda x: parse_coordinates(x)[0])

# Apply the parse_coordinates function to the 'foul' and 'fop' columns
df_cleaned['foul'] = df_cleaned['foul'].apply(parse_coordinates)
df_cleaned['fop'] = df_cleaned['fop'].apply(parse_coordinates)


In [34]:
############## AREA CALCULATION ##############


import pyproj
from shapely.geometry import Polygon
from shapely.ops import transform


def calculate_area(coords):
    # Create a Polygon object from the coordinates
    polygon = Polygon(coords)

    # Calculate the centroid of the polygon
    centroid = polygon.centroid

    # Create a custom LAEA projection centered on the centroid
    custom_projection = f"+proj=laea +lat_0={centroid.y} +lon_0={centroid.x} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

    # Create a transformer for converting coordinates to the custom LAEA projection
    transformer = pyproj.Transformer.from_crs(
        pyproj.CRS("EPSG:4326"),  # WGS 84 (latitude and longitude)
        pyproj.CRS(custom_projection),  # Custom LAEA projection
        always_xy=True
    )

    # Define a function to transform coordinates using the transformer
    def transform_coordinates(x, y):
        return transformer.transform(x, y)

    # Convert the coordinates to the custom LAEA projection
    polygon_laea = transform(transform_coordinates, polygon)

    # Calculate the area in square meters
    area_sqm = polygon_laea.area

    # Convert the area to square feet (1 square meter = 10.764 square feet)
    area_sqft = area_sqm * 10.764

    return area_sqft



### Call Function and add to dataframe
df_cleaned['foul_area_sqft'] = df_cleaned['foul'].apply(calculate_area)
df_cleaned['fop_area_sqft'] = df_cleaned['fop'].apply(calculate_area)

## Calculate the total area of the field and the ratio of foul area to field area
df_cleaned['field_area_sqft'] = df_cleaned['foul_area_sqft'] + df_cleaned['fop_area_sqft']
## Percentage foul area
df_cleaned['foul_area_per'] = df_cleaned['foul_area_sqft'] / df_cleaned['field_area_sqft']
## Fair to Foul Ratio
df_cleaned['fair_to_foul'] = df_cleaned['fop_area_sqft'] / df_cleaned['foul_area_sqft']


In [35]:
############# FENCE DISTANCE CALCULATION #############

from geopy.distance import great_circle
import numpy as np

def interpolate_points(start, end, length_ratio):
    start_np = np.array(start)
    end_np = np.array(end)
    return tuple(start_np + (end_np - start_np) * length_ratio)

def calculate_distances(home_plate, outfield_coords, num_points=90):
    def is_same_point(point1, point2, tolerance=1e-6):
        return abs(point1[0] - point2[0]) < tolerance and abs(point1[1] - point2[1]) < tolerance

    home_plate_lat_lon = (home_plate[1], home_plate[0])
    distances = []

    # Calculate total line length
    total_length = 0
    segments = []
    for i in range(len(outfield_coords) - 1):
        start = outfield_coords[i]
        end = outfield_coords[i + 1]
        if not is_same_point(home_plate, start) and not is_same_point(home_plate, end):
            segment_length = great_circle((start[1], start[0]), (end[1], end[0])).feet
            segments.append((start, end, segment_length))
            total_length += segment_length

    # Calculate the distance between equally spaced points
    spacing = total_length / (num_points - 1)

    # Interpolate points and calculate distances
    current_length = 0
    segment_index = 0
    for i in range(num_points):
        while segment_index < len(segments) - 1 and current_length > segments[segment_index][2]:
            current_length -= segments[segment_index][2]
            segment_index += 1

        start, end, segment_length = segments[segment_index]
        length_ratio = current_length / segment_length
        point = interpolate_points(start, end, length_ratio)
        distance = round(great_circle(home_plate_lat_lon, (point[1], point[0])).feet)
        distances.append(distance)

        current_length += spacing

    return distances

# Calculate distances for each row
df_cleaned['distances'] = df_cleaned.apply(lambda row: calculate_distances(row['home_plate'], row['fop']), axis=1)

# Calculate max, min, and average distances for each row
df_cleaned['max_distance'] = df_cleaned['distances'].apply(max)
df_cleaned['min_distance'] = df_cleaned['distances'].apply(min)
df_cleaned['avg_distance'] = df_cleaned['distances'].apply(lambda distances: sum(distances) / len(distances))
# get the median distance
df_cleaned['median_distance'] = df_cleaned['distances'].apply(lambda distances: np.median(distances))


In [36]:
######## CHECK BLOCK ########

## Check how long the distance list is for each row
df_cleaned['num_distances'] = df_cleaned['distances'].apply(len)

## Print the value counts for the 'num_distances' column
df_cleaned['num_distances'].value_counts()

90    142
Name: num_distances, dtype: int64

In [37]:
######### NOT NECESSARY FOR THIS PROJECT ##########

# ### Get Geolocation of each field based on home plate coordinates and return state and country
# ### This block takes a long time to run - will need to revisit
# ## up to ten minutes

# from geopy.geocoders import Nominatim
# from geopy.exc import GeocoderTimedOut, GeocoderServiceError
# from tqdm import tqdm

# geolocator = Nominatim(user_agent="baseball_field_locator")

# import time

# def get_location_info(lng, lat):
#     try:
#         time.sleep(1)  # Delay for 1 second
#         location = geolocator.reverse((lat, lng), timeout=10)
#         city = location.raw['address'].get('city', None)
#         state = location.raw['address'].get('state', None)
#         return city, state
#     except GeocoderTimedOut:
#         print(f"GeocoderTimedOut error for coordinates: ({lng}, {lat})")
#         return None, None
#     except GeocoderServiceError:
#         print(f"GeocoderServiceError for coordinates: ({lng}, {lat})")
#         return None, None


# # Extract the first coordinate for each field
# df_cleaned['lng'], df_cleaned['lat'] = zip(*df_cleaned['home_plate'].apply(lambda x: x))

# # Wrap the DataFrame apply function with tqdm for progress indication
# tqdm.pandas(desc="Processing coordinates")

# # Get state and country information for each field
# df_cleaned[['city', 'state']] = df_cleaned.progress_apply(lambda row: get_location_info(row['lng'], row['lat']), axis=1, result_type='expand')


In [38]:
## Function to create ranks for each column

def rank_fields(df):
    # Calculate the rank for each category
    df['max_distance_rank'] = df['max_distance'].rank(ascending=False, method='min')
    df['min_distance_rank'] = df['min_distance'].rank(ascending=False, method='min')
    df['avg_distance_rank'] = df['avg_distance'].rank(ascending=False, method='min')
    df['median_distance_rank'] = df['median_distance'].rank(ascending=False, method='min')
    df['field_area_rank'] = df['field_area_sqft'].rank(ascending=False, method='min')
    df['foul_area_rank'] = df['foul_area_sqft'].rank(ascending=False, method='min')
    df['fop_area_per_rank'] = df['fop_area_sqft'].rank(ascending=False, method='min')
    df['ratio_rank'] = df['fair_to_foul'].rank(ascending=False, method='min')

    return df



In [39]:
## Run Function

df_cleaned = rank_fields(df_cleaned)

In [40]:
#### Orienting the map to the home plate location ####

### Find the center of the field
def calculate_centroid(coords):
    x_coords = [coord[0] for coord in coords]
    y_coords = [coord[1] for coord in coords]
    centroid_x = sum(x_coords) / len(coords)
    centroid_y = sum(y_coords) / len(coords)
    return (centroid_x, centroid_y)


## Find the bearing between the home plate and the center of the field
import math

def calculate_bearing(point1, point2):
    lat1, lon1 = math.radians(point1[1]), math.radians(point1[0])
    lat2, lon2 = math.radians(point2[1]), math.radians(point2[0])

    d_lon = lon2 - lon1

    x = math.cos(lat2) * math.sin(d_lon)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(d_lon)

    bearing = math.degrees(math.atan2(x, y))
    bearing = (bearing + 360) % 360  # Normalize the bearing to the range [0, 360)

    return bearing

### Function to classify direction in laymans terms North, South, East, West, ect
def degrees_to_cardinal_direction(degrees):
    directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N']
    index = round(degrees / 45)
    return directions[index]


In [41]:
# Calculate the centroid of the outfield fence coordinates for each row
df_cleaned['fop_centroid'] = df_cleaned['fop'].apply(lambda coords: calculate_centroid(coords[1:]))

# Calculate the bearing between home plate and the centroid for each row
df_cleaned['field_orientation'] = df_cleaned.apply(lambda row: calculate_bearing(row['home_plate'], row['fop_centroid']), axis=1)

# Convert the bearing to a cardinal direction
df_cleaned['field_cardinal_direction'] = df_cleaned['field_orientation'].apply(degrees_to_cardinal_direction)

# rename 'field' to 'park_name'
df_cleaned.rename(columns={'field': 'park_name'}, inplace=True)





### All the geo transformation should take place above this

## starting the process of matching in data from other sources

In [42]:
## output to csv 
df_cleaned.to_csv('data/fields_cleaned.csv', index=False)

In [43]:
# df_cleaned.info()

# # Load the host team info with nickname and team colors
path = 'data/2023_MHSAA_sites.csv'
df_hosts = pd.read_csv(path)

df_parks = df_cleaned

df_hosts.head()

park_df = df_parks
host_df = df_hosts
# df_hosts.info()

# # Merge the host team info with the field info
# df_cleaned = df_cleaned.merge(df_hosts, on='host_team', how='left')

In [44]:
host_df.info()
park_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   park_name             145 non-null    object 
 1   host_team             144 non-null    object 
 2   division              142 non-null    float64
 3   district              128 non-null    float64
 4   region_semi_number    64 non-null     float64
 5   regional_div          78 non-null     float64
 6   region_final_quarter  16 non-null     float64
 7   finals                1 non-null      float64
 8   nickname              143 non-null    object 
 9   color1                143 non-null    object 
 10  color2                143 non-null    object 
 11  color3                14 non-null     object 
dtypes: float64(6), object(6)
memory usage: 13.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 145
Data columns (total 27 columns):
 #   Column            

In [45]:
### Simple Merge, should work because the park_name columns should match exactly
## Do not detroy any data

df_merged = host_df.merge(park_df, on='park_name', how='left')

In [46]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 145
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 145 non-null    object 
 1   host_team                 144 non-null    object 
 2   division                  142 non-null    float64
 3   district                  128 non-null    float64
 4   region_semi_number        64 non-null     float64
 5   regional_div              78 non-null     float64
 6   region_final_quarter      16 non-null     float64
 7   finals                    1 non-null      float64
 8   nickname                  143 non-null    object 
 9   color1                    143 non-null    object 
 10  color2                    143 non-null    object 
 11  color3                    14 non-null     object 
 12  foul                      142 non-null    object 
 13  fop                       142 non-null    object 
 14  notes     

In [47]:
######### FUZZY MATCH FUNCTION - NO LONGER NEEDED #########

# import pandas as pd
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# # Assuming you have two dataframes named 'park_df' and 'host_df'

# # Function to perform fuzzy matching
# def fuzzy_match(row):
#     team = row['team']
#     if pd.isnull(team):
#         return None
#     park_names = park_df['park_name']
#     best_match = process.extractOne(team, park_names, scorer=fuzz.ratio)
#     if best_match[1] >= 80:  # Specify a threshold for matching accuracy
#         return best_match[0]
#     else:
#         return None

# # Add a new column to 'host_df' with the best fuzzy matched park_name
# host_df['fuzzy_matched_park_name'] = host_df.apply(fuzzy_match, axis=1)

# # Merge the dataframes based on the fuzzy matched park_name and team columns
# merged_df = park_df.merge(host_df, left_on='park_name', right_on='fuzzy_matched_park_name', how='left')

# # Print the merged dataframe
# print(merged_df)


In [48]:
df_merged.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 145
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 145 non-null    object 
 1   host_team                 144 non-null    object 
 2   division                  142 non-null    float64
 3   district                  128 non-null    float64
 4   region_semi_number        64 non-null     float64
 5   regional_div              78 non-null     float64
 6   region_final_quarter      16 non-null     float64
 7   finals                    1 non-null      float64
 8   nickname                  143 non-null    object 
 9   color1                    143 non-null    object 
 10  color2                    143 non-null    object 
 11  color3                    14 non-null     object 
 12  foul                      142 non-null    object 
 13  fop                       142 non-null    object 
 14  notes     

In [49]:
### Drop the rows with null values in the 'fop', 'foul' or 'home_plate' columns
df_merged.dropna(subset=['fop', 'foul', 'home_plate'], inplace=True)

In [50]:
df_merged.info()
df = df_merged

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 145
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 142 non-null    object 
 1   host_team                 140 non-null    object 
 2   division                  138 non-null    float64
 3   district                  125 non-null    float64
 4   region_semi_number        63 non-null     float64
 5   regional_div              76 non-null     float64
 6   region_final_quarter      15 non-null     float64
 7   finals                    1 non-null      float64
 8   nickname                  139 non-null    object 
 9   color1                    139 non-null    object 
 10  color2                    139 non-null    object 
 11  color3                    13 non-null     object 
 12  foul                      142 non-null    object 
 13  fop                       142 non-null    object 
 14  notes     

In [51]:
### THIS BLOCK CREATES THE RANKING OF PITCHER VS HITTER FRIENDLY FIELDS
def rank_fields(data):
    # Define weights for each parameter
    weights = {
        'max_distance': -1, # negative weight since longer fences favor pitchers
        'min_distance': 1,  # positive weight since shorter fences favor hitters
        'avg_distance': -.75, # negative weight since longer fences favor pitchers
        'median_distance': -.75, # negative weight since longer fences favor pitchers
        'field_area_sqft': -1,  # negative weight since larger fields favor pitchers
        'fair_to_foul': -1,  # negative weight since larger ratio (more foul territory) favors pitchers
        'foul_area_sqft': -1, # negative weight since larger foul area favors pitchers
        'fop_area_sqft': -1, # negative weight since larger out of play area favors pitchers
    }

    # Standardize features (subtract mean and divide by standard deviation)
    standardized_data = data.copy()
    for column in weights.keys():
        standardized_data[column] = (standardized_data[column] - standardized_data[column].mean()) / standardized_data[column].std()

    # Calculate score for each field
    standardized_data['score'] = standardized_data.apply(lambda row: sum(row[param] * weight for param, weight in weights.items()), axis=1)

    # Save scores to original dataframe
    data['score'] = standardized_data['score']

    # Rank fields based on score (higher scores are more hitter-friendly)
    ranked_fields = data.sort_values('score', ascending=False)

    return ranked_fields

# Suppose 'df' is your DataFrame containing the field data
ranked_fields = rank_fields(df)
print(ranked_fields[['park_name', 'score']])


                              park_name      score
115       Central Lake HS - high_school   8.235868
84                              Hart HS   8.185211
136                         Kingston HS   7.787572
129                           Martin HS   7.206732
132            East Jackson High School   6.653831
..                                  ...        ...
5           Kalamazoo College - college  -8.547677
142  Michigan State - Old College Field  -8.573670
106                   Detroit Osborn HS -13.339288
107                  Laura F. Osborn HS -13.339288
21     Lowell High School - high school -16.047450

[142 rows x 2 columns]


In [52]:
ranked_fields.info()
merged_df = ranked_fields

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 115 to 21
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 142 non-null    object 
 1   host_team                 140 non-null    object 
 2   division                  138 non-null    float64
 3   district                  125 non-null    float64
 4   region_semi_number        63 non-null     float64
 5   regional_div              76 non-null     float64
 6   region_final_quarter      15 non-null     float64
 7   finals                    1 non-null      float64
 8   nickname                  139 non-null    object 
 9   color1                    139 non-null    object 
 10  color2                    139 non-null    object 
 11  color3                    13 non-null     object 
 12  foul                      142 non-null    object 
 13  fop                       142 non-null    object 
 14  notes    

In [53]:
### outpus csv to check
df_merged.to_csv('data/2023_merged_test.csv', index=False)

### OUTPUT JSON TO USE IN MAP
df_merged.to_json('data/html/mhsaa/data/map.json', orient='records')


In [54]:
df_merged.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 145
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 142 non-null    object 
 1   host_team                 140 non-null    object 
 2   division                  138 non-null    float64
 3   district                  125 non-null    float64
 4   region_semi_number        63 non-null     float64
 5   regional_div              76 non-null     float64
 6   region_final_quarter      15 non-null     float64
 7   finals                    1 non-null      float64
 8   nickname                  139 non-null    object 
 9   color1                    139 non-null    object 
 10  color2                    139 non-null    object 
 11  color3                    13 non-null     object 
 12  foul                      142 non-null    object 
 13  fop                       142 non-null    object 
 14  notes     

In [ ]:
df_parks.shape

df_parks.info()

## END BLOCK


In [ ]:
# ## Do a FUZZY MATCH OF DF_HOSTS AND DF_PARKS
# # Debugging step to check for non-string values in host_teams
# for team in host_teams:
#     if not isinstance(team, str):
#         print(f"Non-string value found in host_teams: {team}")

# # Debugging step to check for non-string values in park_names
# for park in park_names:
#     if not isinstance(park, str):
#         print(f"Non-string value found in park_names: {park}")

# # Continue with fuzzy matching if no non-string values are found
# matches = [(team, process.extractOne(team, park_names)) for team in host_teams]


In [ ]:
# ## List the values from the team column

# print(len(df_hosts['team'].unique()))
# df_hosts['team'].unique()

In [ ]:
df_parks = df_cleaned

In [ ]:
parks_df = df_cleaned.copy()

parks_df.info()
host_df.info()

In [ ]:
def merge_dataframes(parks_df, host_df, min_score=90):
    dict_list = []
    unmatched_rows = []
    # Drop NaN values in 'team' and 'park_name' before matching
    host_names = host_df.team.dropna().unique()
    for name in parks_df.park_name.dropna():  # ignore NaN values
        match = match_team(name, host_names, min_score)
        
        # If no match found, add to unmatched_rows and continue to next iteration
        if match[0] == "":
            unmatched_rows.append(name)
            continue

        dict_ = {}
        dict_.update({"park_name_parks" : name})
        dict_.update({"match_name_host" : match[0]})
        dict_.update({"score" : match[1]})
        dict_list.append(dict_)

    merge_table = pd.DataFrame(dict_list)
    
    # Remove duplicates in merge_table, keeping only the row with the highest score
    merge_table = merge_table.sort_values('score', ascending=False).drop_duplicates(['park_name_parks'], keep='first')

    if 'match_name_host' in merge_table.columns:
        merged_df = pd.merge(parks_df, merge_table, left_on='park_name', right_on='park_name_parks', how='left')
        merged_df = pd.merge(merged_df, host_df, left_on='match_name_host', right_on='team', how='left')
    else:
        print("No matches found.")
        merged_df = None

    return merged_df, unmatched_rows


In [ ]:
# merged_df = merged_df.sort_values('score', ascending=False).drop_duplicates(subset=['park_name', 'team'])


In [ ]:
merged_df, unmatched_rows = merge_dataframes(parks_df, host_df, min_score=90)

merged_df.info()

In this function, we first match each team name in host_df with the park names in parks_df. If a match with a similarity score greater than the threshold (85 in your case) is found, we record the match in merge_table. If no match is found, we record the team name in unmatched_rows. After going through all team names, we merge host_df and parks_df based on the matches in merge_table.

The function merge_dataframes returns two objects. The first object, merged_df, is a DataFrame that contains the merged data. The second object, unmatched_rows, is a list of team names in host_df for which no match in parks_df could be found. You can inspect unmatched_rows to see which rows couldn't be matched.

In [ ]:
def merge_dataframes(parks_df, host_df, min_score=90):
    dict_list = []
    unmatched_rows = []
    # Drop NaN values in 'team' and 'park_name' before matching
    host_names = host_df.team.dropna().unique()
    for name in parks_df.park_name.dropna():  # ignore NaN values
        match = match_team(name, host_names, min_score)
        
        # If no match found, add to unmatched_rows and continue to next iteration
        if match[0] == "":
            unmatched_rows.append(name)
            continue

        dict_ = {}
        dict_.update({"park_name_parks" : name})
        dict_.update({"match_name_host" : match[0]})
        dict_.update({"score" : match[1]})
        dict_list.append(dict_)

    merge_table = pd.DataFrame(dict_list)
    
    # Remove duplicates in merge_table, keeping only the row with the highest score
    merge_table = merge_table.sort_values('score', ascending=False).drop_duplicates(['park_name_parks'], keep='first')

    if 'match_name_host' in merge_table.columns:
        merged_df = pd.merge(parks_df, merge_table, left_on='park_name', right_on='park_name_parks', how='left')
        merged_df = pd.merge(merged_df, host_df, left_on='match_name_host', right_on='team', how='left')
    else:
        print("No matches found.")
        merged_df = None

    return merged_df, unmatched_rows



In [ ]:
merged_df.info()

In [ ]:
merged_df.head()




In [ ]:
merged_df.head()

In [ ]:
merged_df.info()

## WORKING HERE DOWN

In [ ]:
## Matching Function to compair host names to park names 

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_host_matches(host_df, parks_df):
    # Initialize empty lists to store the matches and unmatched park names
    matches = []
    unmatched_park_names = []

    # Iterate over each host and district in the host_df
    for host, district in zip(host_df['team'], host_df['district']):
        # Use fuzzy matching to find potential matches in park names
        potential_matches = process.extractBests(host, parks_df['park_name'], scorer=fuzz.token_set_ratio, score_cutoff=80)

        # Store the host, district, and potential matches with their scores
        matches.append({'host': host, 'district': district, 'potential_matches': potential_matches})

        # Check if any strong matches were found
        if len(potential_matches) > 0:
            max_score = max(potential_matches, key=lambda x: x[1])[1]
            if max_score >= 80:
                continue

        # If no strong matches were found, add the park name to unmatched list
        unmatched_park_names.append((host, district))

    # Create a dataframe from the matches list
    matches_df = pd.DataFrame(matches)

    # Count the number of strong matches and unmatched park names
    strong_matches_count = matches_df['potential_matches'].apply(lambda x: sum(match[1] >= 80 for match in x)).sum()
    unmatched_count = len(unmatched_park_names)

    print("Number of strong matches:", strong_matches_count)
    print("Number of unmatched park names:", unmatched_count)
    print("Unmatched park names:", unmatched_park_names)

    return matches_df




In [ ]:
host_df.info()
parks_df.info()

In [ ]:
# Call the function to find host matches
result_df = find_host_matches(host_df, parks_df)

# # Merge the result_df to messe_df on district number
# merged_df = result_df.merge(messe_df, left_on='district', right_on='district number')

# # Select the desired column'region_semi_numbers
# merged_df = merged_df[['host', 'division', 'district', 'region_semi_number', 'region_final_quarter', 'finals', 'potential_matches', 'Plot Note', 'Map Link MHSAA']]
# merged_df.head()
# # merged_df.info()

In [ ]:
### Keep only the field name from the potential matches, ignore if the potential match is empty
merged_df['potential_matches'] = merged_df['potential_matches'].apply(lambda x: x[0][0] if len(x) > 0 else np.nan)


# merged_df['potential_matches'] = merged_df['potential_matches'].apply(lambda x: x[0][0])

# Rename the potential_matches column to park_name

merged_df.rename(columns={'potential_matches': 'park_name'}, inplace=True)


merged_df.head()
merged_df.info()
# # Merge the merged_df to the parks_df on park_name

# parks_df = parks_df.merge(merged_df, on='park_name', how='left')



In [ ]:
## Merge the merged_df to the parks_df on park_name
parks_df = parks_df.merge(merged_df, on='park_name', how='left')

parks_df.head()


In [ ]:
parks_df.info()
parks_df.head()

# host_df.info()
# host_df.head()

# From here down are simple plots to do spot check of data and hold example of polar chart

### FILL IN THE REST OF JSON WITH THE DATA FOR THE 2023 TOURNEY

In [ ]:
### Pull the host team information into the parks_df
parks_df = parks_df.merge(host_df, on='host', how='left')

### Create columns for tournament levels
# District is done, Need regional semi, regional final, quarter final, final_four





parks_df.info()

In [ ]:
import os

# Build the file path using os.path.join
file_path = os.path.join('data', 'html', 'mhsaa', 'data', 'tourney_2023.json')

# Save the dataframe to JSON using the constructed file path
parks_df.to_json(file_path, orient='records')


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# Histogram of the max distance, min distance, average distance, and median distance

fig, ax = plt.subplots(2, 2, figsize=(12, 8))

ax[0, 0].hist(df_cleaned['max_distance'], bins=20)

ax[0, 1].hist(df_cleaned['min_distance'], bins=20)

ax[1, 0].hist(df_cleaned['avg_distance'], bins=20)

ax[1, 1].hist(df_cleaned['median_distance'], bins=20)

ax[0, 0].set_title('Max Distance')
ax[0, 1].set_title('Min Distance')

ax[1, 0].set_title('Average Distance')
ax[1, 1].set_title('Median Distance')

plt.show()


In [ ]:
## Compile a list of fields that are outliers

outlier_fields = df_cleaned[(df_cleaned['max_distance'] > 400) | (df_cleaned['min_distance'] < 200) | (df_cleaned['avg_distance'] > 400) | (df_cleaned['median_distance'] > 400)]

len(outlier_fields)

print(outlier_fields['park_name'].values)



In [ ]:
# Create list of the top and bottom ten from each category

top_ten_max = df_cleaned.sort_values(by='max_distance', ascending=False).head(10)
top_ten_min = df_cleaned.sort_values(by='min_distance', ascending=True).head(10)

top_ten_avg = df_cleaned.sort_values(by='avg_distance', ascending=False).head(10)
top_ten_median = df_cleaned.sort_values(by='median_distance', ascending=False).head(10)

top_ten_field_area = df_cleaned.sort_values(by='field_area_sqft', ascending=False).head(10)

top_ten_foul_area = df_cleaned.sort_values(by='foul_area_sqft', ascending=False).head(10)

top_ten_fop_area = df_cleaned.sort_values(by='fop_area_sqft', ascending=False).head(10)

top_ten_ratio = df_cleaned.sort_values(by='fair_to_foul', ascending=False).head(10)

bottom_ten_ratio = df_cleaned.sort_values(by='fair_to_foul', ascending=True).head(10)

bottom_ten_max = df_cleaned.sort_values(by='max_distance', ascending=True).head(10)
bottom_ten_min = df_cleaned.sort_values(by='min_distance', ascending=False).head(10)
bottom_ten_avg = df_cleaned.sort_values(by='avg_distance', ascending=True).head(10)
bottom_ten_median = df_cleaned.sort_values(by='median_distance', ascending=True).head(10)


### Create and display a dataframe with columns for the top and bottom ten fields for each category

top_bottom_df = pd.DataFrame()

top_bottom_df['top_ten_max'] = top_ten_max['park_name'].values
top_bottom_df['top_ten_min'] = top_ten_min['park_name'].values
top_bottom_df['top_ten_avg'] = top_ten_avg['park_name'].values

top_bottom_df['top_ten_median'] = top_ten_median['park_name'].values
top_bottom_df['top_ten_field_area'] = top_ten_field_area['park_name'].values
top_bottom_df['top_ten_foul_area'] = top_ten_foul_area['park_name'].values
top_bottom_df['top_ten_fop_area'] = top_ten_fop_area['park_name'].values
top_bottom_df['top_ten_ratio'] = top_ten_ratio['park_name'].values

top_bottom_df['bottom_ten_ratio'] = bottom_ten_ratio['park_name'].values
top_bottom_df['bottom_ten_max'] = bottom_ten_max['park_name'].values
top_bottom_df['bottom_ten_min'] = bottom_ten_min['park_name'].values
top_bottom_df['bottom_ten_avg'] = bottom_ten_avg['park_name'].values
top_bottom_df['bottom_ten_median'] = bottom_ten_median['park_name'].values


top_bottom_df.head(10)




In [ ]:
### NEW WITH AUTO SCALING

def calculate_max_y(data, num_bins=36, level_filter=None):
    count_by_orientation = process_data(data, level_filter)
    
    # Compute the histogram
    bin_counts = np.zeros(num_bins)
    
    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count
    
    return max(bin_counts)


def create_polar_chart(data, num_bins=36, level_filter=None, y_min=-20, background_color='#2b2b2b', color_map=plt.cm.viridis, bar_alpha=0.8):
    count_by_orientation = process_data(data, level_filter)
    
    # Compute the histogram
    bin_edges = np.linspace(0.0, 2 * np.pi, num_bins + 1)
    bin_counts = np.zeros(num_bins)
    
    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count
    
    bin_width = 2 * np.pi / num_bins

    # Set plot size
    plt.figure(figsize=(10, 10))

    ax = plt.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_offset(np.pi / 2)

    ax.set_facecolor('#808080')
    # ax.set_yscale('log')

    # Set dark background
    ax.set_facecolor(background_color)
    plt.gca().set_rlabel_position(22.5)
    y_max = calculate_max_y(data, num_bins=num_bins, level_filter=level_filter) + 5
    ax.set_ylim(y_min, y_max)  # Adjust based on max count

    # Add bars for negative values
    zero_height_bars = ax.bar(bin_edges[:-1], np.abs(ax.get_ylim()[0]), width=bin_width, bottom=0.0, color='k', alpha=0.3)

    bars = ax.bar(bin_edges[:-1], bin_counts, width=bin_width, bottom=0)
    
    # Use custom colors and opacity
    for r, bar in zip(bin_counts, bars):
        bar.set_facecolor(color_map(r / max(bin_counts)))
        bar.set_alpha(bar_alpha)

    plt.show()


In [ ]:
##### CALL AUTO ADJUSTING CHART #####


## NEW PERAMS


# Call your function
create_polar_chart(
    data, 
    num_bins=30, 
    # level_filter="level1", 
    y_min=0, 
    background_color='#2b2b2b', 
    color_map=plt.cm.viridis, 
    bar_alpha=0.8
)


In [ ]:
## NEW CHAT GPT CODE

def create_polar_chart(data, num_bins=36, level_filter=None, y_min=-20, y_max=130, background_color='#2b2b2b', color_map=plt.cm.viridis, bar_alpha=0.8):
    count_by_orientation = process_data(data, level_filter)
    
    # Compute the histogram
    bin_edges = np.linspace(0.0, 2 * np.pi, num_bins + 1)
    bin_counts = np.zeros(num_bins)
    
    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count
    
    bin_width = 2 * np.pi / num_bins

    # Set plot size
    plt.figure(figsize=(10, 10))

    ax = plt.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_offset(np.pi / 2)

    ax.set_facecolor('#808080')
    # ax.set_yscale('log')

    # Set dark background
    ax.set_facecolor(background_color)
    plt.gca().set_rlabel_position(22.5)
    ax.set_ylim(y_min, y_max)  # Adjust based on max count

    # Add bars for negative values
    zero_height_bars = ax.bar(bin_edges[:-1], np.abs(ax.get_ylim()[0]), width=bin_width, bottom=0.0, color='k', alpha=0.3)

    bars = ax.bar(bin_edges[:-1], bin_counts, width=bin_width, bottom=0)
    
    # Use custom colors and opacity
    for r, bar in zip(bin_counts, bars):
        bar.set_facecolor(color_map(r / max(bin_counts)))
        bar.set_alpha(bar_alpha)

    plt.show()


In [ ]:
### Create a polar chart showing the direction of all the tournment fields


import numpy as np
import matplotlib.pyplot as plt

# create a function to process the data, counting the orientations and filtering by level.

from collections import defaultdict

def process_data(data, level_filter=None):
    count_by_orientation = defaultdict(int)
    
    for record in data:
        if level_filter is None or record['level'] == level_filter:
            orientation = round(record['field_orientation'])
            count_by_orientation[orientation] += 1

    return count_by_orientation

def create_polar_chart(data, num_bins=36, level_filter=None):
    count_by_orientation = process_data(data, level_filter)
    
    # Compute the histogram
    bin_edges = np.linspace(0.0, 2 * np.pi, num_bins + 1)
    bin_counts = np.zeros(num_bins)
    
    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count
    
    bin_width = 2 * np.pi / num_bins

    # Set plot size
    plt.figure(figsize=(10, 10))

    ax = plt.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_offset(np.pi / 2)

    ax.set_facecolor('#808080')
    ###
    # ax.set_yscale('log')

    
    # # Set dark background
    ax.set_facecolor('#2b2b2b')
    plt.gca().set_rlabel_position(22.5)
    ax.set_ylim(-20, 130)  # Adjust based on max count

    # Add bars for negative values
    zero_height_bars = ax.bar(bin_edges[:-1], np.abs(ax.get_ylim()[0]), width=bin_width, bottom=0.0, color='k', alpha=0.3)

    bars = ax.bar(bin_edges[:-1], bin_counts, width=bin_width, bottom=0)
    
    # Use custom colors and opacity
    for r, bar in zip(bin_counts, bars):
        bar.set_facecolor(plt.cm.viridis(r / max(bin_counts)))
        # bar.set_facecolor(plt.cm.plasma(r / max(bin_counts)))
        bar.set_alpha(0.8)

    plt.show()

In [ ]:
create_polar_chart(data, num_bins=50, level_filter=None)